In [3]:
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
from model import *
import torch
from data_loader import data_loader


Using backend: pytorch


In [16]:
dataset = data_loader('../DBLP')
edge_dict = {}

for i, meta_path in dataset.links['meta'].items():
    edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = (torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]), torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]))
np.nonzero(dataset.labels_train['mask'])[0].max()

4051

In [17]:
print(dataset.nodes['count'][0])
node_dict = {}
for i, node_count in dataset.nodes['count'].items():
    print(i, node_count)
    node_dict[str(i)] = node_count

4057
0 4057
1 14328
2 7723
3 20


In [71]:
G = dgl.heterograph(edge_dict, num_nodes_dict = node_dict)

KeyError: '0'

In [61]:
node_dict = {}
edge_dict = {}
for ntype in G.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in G.etypes:
    edge_dict[etype] = len(edge_dict)
    G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long) * edge_dict[etype] 


In [68]:
for ntype in G.ntypes:
    emb = nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), 256), requires_grad = False)
    nn.init.xavier_uniform_(emb)
    G.nodes[ntype].data['inp'] = emb

DGLError: Node type "ntype" does not exist.

In [ ]:
G.